In [70]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/?/*
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

In [71]:
iOption0 = 0
is_sigmalised = 1
sim = 1
new = 1

Min_Entries = 100

In [72]:
isigma = 2
N_files = 1
N_layers = 16
N_hists = 4
N_phi = 6
N_ilyaers = 1
is_e = 1
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
phi_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
phi_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if is_sigmalised: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/rg/"
file_names="my-27th_QA_19200_350runs" #my-25th_QA_19197_1000runs
if new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new8_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [73]:
if sim :
    file_path="../sim/"
    file_names="my-kek"

    #phi_bins = [96,97,98,99,100,100,101,101,102,103,104,105]

In [74]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if not new: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    else: infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_emc_read0 = []
    for ilayer in range(N_layers):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

../sim/my-kek0.root


In [75]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.5,0.8,1.2,1.6,2.0,2.4,3.0,3.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.35, 0.65, 1.0, 1.4, 1.8, 2.2, 2.7, 3.4, 4.35]) 
pt_boarders =  array('d', [0.2, 0.5, 0.8, 1.2, 1.6, 2.0, 2.4, 3.0, 3.8, 4.9])


In [76]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]
arms = N_layers/8

In [77]:
def emc_fitter(InHist, iphi, seed=1, ilayer=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*ilayer*100
    iilayer = int(ilayer/arms)
    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    central_bean_prev = (0.001*phi_bins[2*iphi]-0.0005-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(ilayer)+"/"+str(iphi)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{ilayer}_{iphi}",phi_bins[2*iphi],phi_bins[2*iphi],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        if emc_var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = 0.5* (pt_boarders[p]+pt_boarders[p+1])
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.

        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit[int(ilayer%2)][0]+central_bean_prev, boards_fit[int(ilayer%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{ilayer}_{iphi}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{ilayer}_{iphi}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(ilayer%2)][0], boards_tot[int(ilayer%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        g_gaus.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
        g_gaus2.SetParameter(1,0.5*(boards_fit2[iilayer][0]+boards_fit2[iilayer][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[iilayer][1]-boards_fit2[iilayer][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(ilayer%2)][1]/100,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if iphi>-1:
            total.SetParLimits(1,boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        if iOption0<4 and is_sigmalised==0:
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))
        else:
            Mean_local_err.append(total.GetParError(1)+ (0.15-0.005*p)*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+(0.15-0.005*p)*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]


In [78]:
def emc_centr_fitter(ilayer=1,InHist=hists_emc_read, phibins=[0,1,2,3,4,5], sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for iFile in range(N_files):
        out_hists1 = []
        for iphi in phibins:
            out_hists1.append(emc_fitter(InHist[iFile][ilayer][iOption0+is_sigmalised*2], iphi, iFile, ilayer, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [79]:
#Ntr = N_files
Ntr = N_layers
pool = mp.Pool(Ntr)
#output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
with pool:
    output_array = pool.map(emc_centr_fitter, [i for i in range(Ntr)])
pool.close()

In [80]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(N_files):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layers):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for iphi in range(N_phi):
            Means0.append(output_array[ilayer][ifile][iphi][0])
            Mean_errs0.append(output_array[ilayer][ifile][iphi][1])
            Sigmas0.append(output_array[ilayer][ifile][iphi][2])
            Sigma_errs0.append(output_array[ilayer][ifile][iphi][3])
            Int_FG0.append(output_array[ilayer][ifile][iphi][4])
            Int_BG0.append(output_array[ilayer][ifile][iphi][5])
            pt_arr0.append(output_array[ilayer][ifile][iphi][6])
            pt_arr_err0.append(output_array[ilayer][ifile][iphi][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [81]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{iphi}_{ilayer}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.5)
                fit_functions1[-1].SetParLimits(1,0.001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [82]:
g_sigma = []
for ifile in range(N_files):
    g_sigma_f = []
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        g_sigma_layer = []
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_sigma_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Sigmas[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Sigma_errs[ifile][ilayer][iphi]))
            g_sigma_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_sigma_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if iOption0 == 0 and ilayer%2==0 and (iphi == 0 or iphi == N_phi-1): 
                g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","",0.2,1.2)
            else: g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","")
            g_sigma_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_sigma_f.append(g_sigma_layer)
    g_sigma.append(g_sigma_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

In [83]:
print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_ilayer in fit_functions_ifile:
        print("{")
        for fits in fit_functions_ilayer:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[1][16][6][3]={
{
{
{0.012779, 0.001341, 10.23},
{0.010545, 0.0, -4.792628},
{0.0, 20.0, -3.026892},
{0.709515, 0.0, -20.800508},
{0.069555, 0.001346, 10.23},
{0.0, 20.0, -3.578482},
},
{
{0.189218, 0.001174, 10.23},
{0.0, 1.768491, -0.542946},
{0.0, 1.822153, -0.770093},
{0.0, 2.15854, -0.809825},
{0.0, 1.703232, -0.467541},
{2e-06, 2.024972, -0.660746},
},
{
{0.0, 1.467997, -0.587343},
{0.069817, 0.001233, 10.23},
{0.346109, 19.999999, -3.983881},
{0.000541, 1.146406, -0.087393},
{0.64188, 3e-06, -13.258945},
{0.0, 2.745541, -1.179762},
},
{
{0.0, 2.951404, -1.258362},
{0.0, 1.891183, -1.133069},
{0.01352, 0.0012, 10.229999},
{0.01597, 0.002119, 10.226118},
{0.0, 1.388796, -0.799401},
{0.0, 1.861332, -1.120873},
},
{
{0.000708, 1.134861, -0.169736},
{0.006176, 1.007336, -0.054647},
{0.0, 2.454549, -1.164785},
{0.000445, 1.042808, -0.074842},
{0.033037, 0.96338, -0.049441},
{2e-06, 1.476807, -0.534238},
},
{
{0.827475, 3.5e-05, -20.171831},
{0.926872, 0.086027, -0.24848

In [84]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0==0:
                    fit_functions1[-1].SetParLimits(2,-5.126,-0.23)
                    if ilayer%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-2.0,-1)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,0.1,1.3)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,1,2.0)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-1.3,-0.1)
                    if iphi==3 and ilayer ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        fit_functions1[-1].SetParameter(1,1.6)
                        fit_functions1[-1].SetParameter(2,-0.9)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [85]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,225*N_layers)
    c3.Divide(2,int(N_layers/2))
    g_mean = [[[[]]*N_phi]*N_layers]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_mean[ifile][ilayer][iphi].append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Means[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Mean_errs[ifile][ilayer][iphi]))
            
            Format_Graph(g_mean[ifile][ilayer][iphi][-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            g_mean[ifile][ilayer][iphi][-1].Draw('P')
            if not is_sigmalised: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.7,1)
            else: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.5,1)
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][ilayer][iphi][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [86]:
if not is_sigmalised:
    print(f"mean_pt_params[{N_files}][{N_layers}][{N_phi}]="+"{")
    for fit_functions_file in fit_mean_functions:
        print("{")
        for fit_functions_syst in fit_functions_file:
            print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
        print("},")
    print("};")
else:
    print("const float "+iOptames[iOption0]+f"_sMean_pt_params[{N_files}][{N_layers}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        iilayer = 0
        for fits in fit_functions_ifile:
            print("{"+f"{round(fits[3-int(iilayer%2)].GetParameter(0),6)}, {round(fits[3-int(iilayer%2)].GetParameter(1),6)}, {round(fits[3-int(iilayer%2)].GetParameter(2),6)}"+"},")
            iilayer+=1
        print("},")      
    print("};")        
    print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        for fit_functions_ilayer in fit_functions_ifile:
            print("{")
            for fits in fit_functions_ilayer:
                print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
            print("},") 
        print("},")      
    print("};")        

const float the_sMean_pt_params[1][16][3]={
{
{-0.01823, -0.022416, 1.104186},
{-0.155543, -0.386791, -1.729259},
{-0.00035, -0.00035, 0.0},
{0.012523, 0.018777, 1.81598},
{0.16209, 0.163111, -0.500441},
{0.035325, -0.492801, -2.056209},
{0.05235, 0.239655, -2.173962},
{0.024821, -0.362618, -2.025782},
{0.067114, 0.068189, 0.121978},
{-0.011369, 0.000923, 4.618598},
{-0.122904, 0.114123, 0.230041},
{0.094869, -0.868397, -2.837177},
{0.005484, 0.002744, 2.978146},
{0.043588, 0.050948, 1.258111},
{0.147332, -0.575637, -1.561232},
{-0.041092, 0.009457, 2.951904},
},
};
sigma_pt_params[1][16][6][3]={
{
{
{-0.011296, 0.028894, 2.829966},
{0.049613, 0.054451, 1.16147},
{-0.088046, 0.032738, 2.196004},
{-0.01823, -0.022416, 1.104186},
{-0.085062, -0.074871, 1.189155},
{-0.079404, -0.066902, 1.549596},
},
{
{0.0446, -0.936837, -2.873295},
{-0.081912, -0.188341, -2.094153},
{-0.155543, -0.386791, -1.729259},
{-0.196035, -0.196006, 0.102609},
{-0.219377, -0.186446, 1.103323},
{-0.302912, -0.3014

In [87]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for iphi in range(N_phi):
    if not is_sigmalised: dphi.append( round(0.001*phi_bins[2*iphi]-0.0005-0.1,5))
    else: dphi.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
    dphi_err.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParameter(0))
            dynamic_dphi_err_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1])
[[array('d', [-0.011295641526623424, 0.04961332839230827, -0.088045839851687, -0.018229527261322136, -0.08506170468869441, -0.07940442169092596]), array('d', [0.04459957142027225, -0.081912213056492, -0.15554295074896934, -0.19603497409958842, -0.21937665157027122, -0.3029124725395663]), array('d', [0.060823806413600766, 0.029856679752177324, 0.006464167969155189, -0.00035023883276848606, -0.09533258835585401, -0.055570296096336076]), array('d', [0.04094423451938771, 0.01288539304263056, 0.012523041218893464, -0.023397362501356076, -0.07551540296727846, -0.07395568574185739]), array('d', [0.0703782944296033, 0.048689639642918746, 0.08002153731033436, 0.16208961183721493, -0.19324336128000869, 0.07873461702248136]), array('d', [0.03629171381340262, 0.048372128161422744, 0.03532472417597194, -0.048132980689991424, -0.0312176209707813, -0.03577233080460245]), array('d', [-0.0750450825471408, -0.007152383578484123, 0.0218560153124474, 0

In [88]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 2, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "mean [rad]", 1.1, 1.1, 0.06, 0.05, "",0.2,0.2,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, dynamic_dphi[ifile][ilayer],dphi_err,dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_mean_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_mean_phi[ifile][ilayer][iphi][-1].Fit("pol1","Q","")
            g_mean_phi[ifile][ilayer][iphi][-1].GetFunction("pol1").SetLineColor(colors[ilayer%4])
            g_mean_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [89]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_functions[ifile][ilayer][iphi].Eval(pt_point)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
            dynamic_dphi_err_ilayer.append((fit_functions[ifile][ilayer][iphi].GetParError(0)*2)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [0.09232125425027395, 0.004125057719227162, 1.7224401683616368, 0.2775598316383634, 0.11486828698010207, 1.3072778098730704]), array('d', [0.2870933819042996, 1.0061332225326862, 0.9253654961490847, 1.0746345038509153, 1.006237307073363, 1.0861566366337876]), array('d', [0.5245698666464363, 0.13183279265072237, 1.473748178581387, 0.5262518214186132, 0.30766089297737326, 0.7295650789517165]), array('d', [5.411774039974974, 3.6919144007217497, 0.7337996566242808, 1.266200343375719, 3.203405386728218, 3.655865050917037]), array('d', [0.8781517486952419, 0.8302819407176671, 1.1540693125482056, 0.8459306874517946, 0.8189462023825499, 0.9517136057641981]), array('d', [0.861542125941093, 1.044135479515919, 1.0517517473435205, 0.9482482526564796, 1.0652785490404115, 1.0109322971062098]), array('d', [1.0566390410456048, 0.959334472822766, 0.9868888437849699, 1.0131111562150303, 0.9930183834152199, 0.9878862791231611]), array('d', [0.9979883298710671, 1.339749298435965, 1.1864583739

In [90]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "sigma [rad]", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, sigma_dynamic_dphi[ifile][ilayer],dphi_err,sigma_dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_sigma_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma_phi[ifile][ilayer][iphi][-1].Fit("pol2","Q","",)
            g_sigma_phi[ifile][ilayer][iphi][-1].GetFunction("pol2").SetLineColor(colors[ilayer%4])
            g_sigma_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [91]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_layers}][3]="+"\n{")
for fit_functions_ifile in fit_functions:
    print("    {")
    for fits in fit_functions_ifile:
        print("        {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("    },")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][3]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(0),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(1),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("    },")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][2]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_mean_phi[ifile][ilayer][0][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][ilayer][0][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("    },")
print("};")



const float the_sigma_pt_params[1][16][3]=
{
    {
        {0.354757, 10.0, -11.9137},
        {0.0, 1.990347, -0.789959},
        {0.173325, 10.573202, -2.035637},
        {0.014745, 0.001659, 10.228059},
        {0.000222, 1.748679, -0.619813},
        {0.505203, 0.436411, -0.261802},
        {0.878368, 0.069669, -0.561634},
        {0.503443, 0.363381, -9.302234},
        {0.614244, 0.47463, -0.100971},
        {0.001713, 1.141094, -0.126757},
        {0.523019, 0.693836, -0.460104},
        {0.00997, 1.307972, -0.22898},
        {0.00034, 1.118593, -0.130448},
        {0.0, 1.812719, -0.626451},
        {0.0, 3.872019, -1.398705},
        {0.0, 1.856728, -0.624849},
    },
};
const float the_sigma_the_params[1][16][3]=
{
    {
        {3.067822362787464e+16, -1452789526143.579, -5.5281497118048026e+17},
        {3.067822362787464e+16, -1452789526143.579, -5.5281497118048026e+17},
        {3.067822362787464e+16, -1452789526143.579, -5.5281497118048026e+17},
        {3.06782236278746

In [92]:
outrootfile = root.TFile(f"output/dynamic_delta_fits/dynamicdeltasigma{iOption0}{sim}.root","RECREATE")
for g_sigma_f in g_sigma:
    for g_sigma_layer in g_sigma_f:
        for g_sigma_phi in g_sigma_layer:
            g_sigma_phi.Write()